# Notebook Contents

1. ### Testing `BubbleBurster` instantiation
2. ### Testing evalmetrics - `NoveltyMetric`
3. ### Testing evalmetrics - `SerendipityMetric`
4. ### Testing evalmetrics - `DiversityMetric`
5. ### Testing **all** evalmetrics

# Setup

In [1]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../t-recs/')
from trecs.models import ContentFiltering
from trecs.metrics import *
from trecs.random import Generator
from trecs.components import Users

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

random_state = np.random.seed(42)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

ratings_df = pd.read_csv('../data/ml-100k/u.data', 
    sep="\t", 
    names=['UserID', 'MovieID', 'Rating', 'Timestamp']
)

movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies_df = pd.read_csv('../ml-100k/u.item', sep="|", names=movie_cols, encoding='latin')

# display(movies_df.head(2))
# print(movies_df.shape)

FileNotFoundError: [Errno 2] File ../ml-100k/u.data does not exist: '../ml-100k/u.data'

In [ ]:
from sklearn.cluster import KMeans

def get_topic_clusters(binary_ratings_matrix, n_attrs:int=100, nmf_solver:str="mu"):
    """
    Creates clusters of movies based on their genre.
    Inputs:
        binary_ratings_matrix: a binary matrix of users and movies
        n_attrs: number of attributes to use in NMF
        nmf_solver: solver to use in NMF
    Outputs:
        clusters: a list of cluster assignments
    """
    # Create topic clusters
    #create co-occurence matrix from binary_interaction_matrix
    co_occurence_matrix = binary_ratings_matrix.T @ binary_ratings_matrix
    co_occurence_matrix

    # Matrix factorize co_occurence_matrix to get embeddings
    nmf_cooc = NMF(n_components=n_attrs, solver=nmf_solver)
    W_topics = nmf_cooc.fit_transform(co_occurence_matrix)

    # cluster W_topics
    kmeans = KMeans(n_clusters=100, random_state=random_state).fit(W_topics)

    # assign nearest cluster to observation
    cluster_ids = kmeans.predict(W_topics)

    return cluster_ids

In [ ]:
from sklearn.decomposition import NMF

binary_ratings_df = ratings_df.drop(columns=['Timestamp'])
binary_ratings_df.loc[binary_ratings_df['Rating'] > 0, 'Rating'] = 1

# turn dataframe into matrix where each movie is a column and each user is a row
binary_ratings_matrix = binary_ratings_df.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0).to_numpy()

from lightfm.cross_validation import random_train_test_split
from scipy import sparse

# split data into train and test sets
train_interactions, test_interactions = random_train_test_split(sparse.csr_matrix(binary_ratings_matrix), test_percentage=0.2, random_state=random_state)
train_interactions = train_interactions.toarray()
test_interactions = test_interactions.toarray()

n_attrs=100
nmf = NMF(n_components=n_attrs, solver="mu")
user_representation = nmf.fit_transform(binary_ratings_matrix)
item_representation = nmf.components_
print(user_representation.shape, item_representation.shape)

num_topics = None
item_topics = get_topic_clusters(binary_ratings_matrix, n_attrs=n_attrs, nmf_solver="mu")
user_topic_history = None
item_count = None

from wrapper.models.bubble import BubbleBurster

users = Users(size=(943,100), repeat_interactions=False)

# Testing `BubbleBurster` model instantiation

In [ ]:
recsys = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
print(recsys.num_topics)
print(recsys.item_topics.shape)
print(recsys.user_topic_history.shape)
print(np.unique(recsys.user_topic_history))
print(recsys.item_count.shape) 

In [ ]:
recsys.add_metrics(MSEMeasurement(), InteractionSpread(), AverageFeatureScoreRange())
print("These are the current metrics:")
print(recsys.metrics)

In [ ]:
# now we run the model
recsys.run(timesteps=1)
measurements = recsys.get_measurements()

**-> Model successfully runs for 1 timestep**

In [ ]:
keys = recsys.__dict__.keys()
vals = recsys.__dict__.values()

for k, v in zip(keys,vals):
    print(k)

In [ ]:
def state_update(recommender, item_count, user_topic_history, item_topics):
    items_shown = recommender.items_shown
    for i in range(items_shown.shape[0]):
        items_shown_val, items_shown_count = np.unique(items_shown[i,:], return_counts=True)
        item_count[0, items_shown_val] += 1
        topics_shown = item_topics[items_shown_val]
        topics_shown_val, topics_shown_count = np.unique(topics_shown, return_counts=True)
        user_topic_history[i, topics_shown_val] += topics_shown_count
        if (sum(items_shown_count) != 10):
            print("DUPLICATE ITEMS IN SLATE", items_shown_count)
            break
    return item_count, user_topic_history

In [ ]:
test_item_count = np.zeros((1,recsys.num_items))
test_user_topic_history = np.zeros((recsys.num_users, recsys.num_topics))

test_item_count, test_user_topic_history = state_update(recsys, test_item_count, test_user_topic_history, item_topics)

In [ ]:
print(np.array_equal(recsys.item_count, test_item_count))
print(np.array_equal(recsys.user_topic_history, test_user_topic_history))

**-> Two values are equal after 1 iteration**

In [ ]:
# now we run the model
recsys.run(timesteps=1)
measurements = recsys.get_measurements()

In [ ]:
test_item_count, test_user_topic_history = state_update(recsys, test_item_count, test_user_topic_history, item_topics)
print(np.array_equal(recsys.item_count, test_item_count))
print(np.array_equal(recsys.user_topic_history, test_user_topic_history))

**-> The two values are equal after a second iteration**

In [ ]:
for key in measurements.keys():
    print(key, measurements[key])

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
bubble.add_metrics(MSEMeasurement(), InteractionSpread(), AverageFeatureScoreRange())
print("These are the current metrics:")
print(recsys.metrics)

In [ ]:
bubble.startup_and_train(timesteps=100)

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

# Testing evalmetrics - `NoveltyMetric`

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
from wrapper.metrics import NoveltyMetric

bubble.add_metrics(MSEMeasurement(), NoveltyMetric())
print("These are the current metrics:")
print(bubble.metrics)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
# x = bubble.users.actual_user_scores.filter_by_index(bubble.items_shown).reshape(bubble.num_users, bubble.num_items_per_iter)#[:5,:10]#.shape
# y = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1)#.shape
# np.array_equal(x, y)

In [ ]:
# t = bubble.users.get_actual_user_scores().get_item_scores(bubble.items_shown)
# # t.filter_by_index(bubble.items_shown)
# t[:,:]

In [ ]:
slate_items_self_info = bubble.item_count[bubble.items_shown]
slate_items_self_info = (-1) * np.log(np.divide(slate_items_self_info, bubble.num_users))
print(slate_items_self_info.shape)

slate_items_pred_score = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1)#.shape
print(slate_items_pred_score.shape)

slate_novelty = np.multiply(slate_items_self_info, slate_items_pred_score)
print(slate_novelty.shape)

slate_novelty = np.sum(slate_novelty, axis=1)
print(slate_novelty.shape)

novelty = np.mean(slate_novelty)
print(novelty)

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

In [ ]:
print("Is novelty equal? ", measurements['novelty_metric'][-1] == novelty)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
slate_items_self_info = bubble.item_count[bubble.items_shown]
slate_items_self_info = (-1) * np.log(np.divide(slate_items_self_info, bubble.num_users))
print(slate_items_self_info.shape)

slate_items_pred_score = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1)#.shape
print(slate_items_pred_score.shape)

slate_novelty = np.multiply(slate_items_self_info, slate_items_pred_score)
print(slate_novelty.shape)

slate_novelty = np.sum(slate_novelty, axis=1)
print(slate_novelty.shape)

novelty = np.mean(slate_novelty)
print(novelty)

In [ ]:
print("Is novelty equal? ", measurements['novelty_metric'][-1] == novelty)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
slate_items_self_info = bubble.item_count[bubble.items_shown]
slate_items_self_info = (-1) * np.log(np.divide(slate_items_self_info, bubble.num_users))
print(slate_items_self_info.shape)

slate_items_pred_score = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1)#.shape
print(slate_items_pred_score.shape)

slate_novelty = np.multiply(slate_items_self_info, slate_items_pred_score)
print(slate_novelty.shape)

slate_novelty = np.sum(slate_novelty, axis=1)
print(slate_novelty.shape)

novelty = np.mean(slate_novelty)
print(novelty)

In [ ]:
print("Is novelty equal? ", measurements['novelty_metric'][-1] == novelty)

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

# Testing evalmetrics - `SerendipityMetric`

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
from wrapper.metrics import SerendipityMetric

bubble.add_metrics(MSEMeasurement(), SerendipityMetric())
print("These are the current metrics:")
print(bubble.metrics)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

In [ ]:
# Indices for the items shown
items_shown = bubble.items_shown
print(items_shown.shape)
# Scores for the items shown
user_scores = bubble.users.actual_user_scores.value
print(user_scores.shape)
# Scores for just the shown items that have a score greater than 0
user_scores_items_shown = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1) > 0
print(user_scores_items_shown.shape)
# Topics that correspond to each item shown
topics_shown = bubble.item_topics[bubble.items_shown]
print(topics_shown.shape)
"""
Need to update the below 2 lines depending on how user_topic_history is implemented in the wrapper class
"""
# Boolean matrix where value=1 if the topic shown is not in the user history, otherwise value=0
new_topics = np.apply_along_axis(np.isin, 1, topics_shown, bubble.user_topic_history, invert=True)
print(new_topics.shape)

# # calculate serendipity for all items presented to each user
items_shown_serendipity = np.multiply(new_topics, user_scores_items_shown)
print(items_shown_serendipity.shape)
# Calculate average serendipity - average serendipity by slate AND users
serendipity = np.mean(items_shown_serendipity)#np.sum(np.multiply(new_topics, user_scores_items_shown)) / bubble.num_users
print(serendipity)
# # to complete the measurement, call `self.observe(metric_value)`

In [ ]:
print("Is serendipity equal? ", measurements['serendipity_metric'][-1] == serendipity)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
# Indices for the items shown
items_shown = bubble.items_shown
print(items_shown.shape)
# Scores for the items shown
user_scores = bubble.users.actual_user_scores.value
print(user_scores.shape)
# Scores for just the shown items that have a score greater than 0
user_scores_items_shown = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1) > 0
print(user_scores_items_shown.shape)
# Topics that correspond to each item shown
topics_shown = bubble.item_topics[bubble.items_shown]
print(topics_shown.shape)
"""
Need to update the below 2 lines depending on how user_topic_history is implemented in the wrapper class
"""
# Boolean matrix where value=1 if the topic shown is not in the user history, otherwise value=0
new_topics = np.apply_along_axis(np.isin, 1, topics_shown, bubble.user_topic_history, invert=True)
print(new_topics.shape)

# # calculate serendipity for all items presented to each user
items_shown_serendipity = np.multiply(new_topics, user_scores_items_shown)
print(items_shown_serendipity.shape)
# Calculate average serendipity - average serendipity by slate AND users
serendipity = np.mean(items_shown_serendipity)#np.sum(np.multiply(new_topics, user_scores_items_shown)) / bubble.num_users
print(serendipity)
# # to complete the measurement, call `self.observe(metric_value)`

In [ ]:
print("Is serendipity equal? ", measurements['serendipity_metric'][-1] == serendipity)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
# Indices for the items shown
items_shown = bubble.items_shown
print(items_shown.shape)
# Scores for the items shown
user_scores = bubble.users.actual_user_scores.value
print(user_scores.shape)
# Scores for just the shown items that have a score greater than 0
user_scores_items_shown = np.take_along_axis(bubble.users.actual_user_scores.value, bubble.items_shown, axis=1) > 0
print(user_scores_items_shown.shape)
# Topics that correspond to each item shown
topics_shown = bubble.item_topics[bubble.items_shown]
print(topics_shown.shape)
"""
Need to update the below 2 lines depending on how user_topic_history is implemented in the wrapper class
"""
# Boolean matrix where value=1 if the topic shown is not in the user history, otherwise value=0
new_topics = np.apply_along_axis(np.isin, 1, topics_shown, bubble.user_topic_history, invert=True)
print(new_topics.shape)

# # calculate serendipity for all items presented to each user
items_shown_serendipity = np.multiply(new_topics, user_scores_items_shown)
print(items_shown_serendipity.shape)
# Calculate average serendipity - average serendipity by slate AND users
serendipity = np.mean(items_shown_serendipity)#np.sum(np.multiply(new_topics, user_scores_items_shown)) / bubble.num_users
print(serendipity)
# # to complete the measurement, call `self.observe(metric_value)`

In [ ]:
print("Is serendipity equal? ", measurements['serendipity_metric'][-1] == serendipity)

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

In [ ]:
new_topics.size == np.count_nonzero((new_topics==0) | (new_topics==1))

# Testing evalmetrics - `DiversityMetric`

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    # actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
from wrapper.metrics import DiversityMetric

bubble.add_metrics(MSEMeasurement(), DiversityMetric())
print("These are the current metrics:")
print(bubble.metrics)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
for key in measurements.keys():
    print(key, measurements[key])

In [ ]:
from itertools import combinations

# Getting all possible 2-item combinations (the indices) for the items in a slate
combos = combinations(np.arange(bubble.num_items_per_iter), 2)
items_shown = bubble.items_shown

topic_similarity = np.zeros(bubble.num_users)

stop = 0
for i in combos:
    # topic_similarity is equal to the number of 2-item combinations in which the items' topics are the same
    item_pair = items_shown[:, i]
    topic_pair = bubble.item_topics[item_pair]
    topic_similarity += (topic_pair[:,0] == topic_pair[:,1])

slate_diversity = 1 - ((1 / (bubble.num_items_per_iter * (bubble.num_items_per_iter-1))) * topic_similarity)
diversity = np.mean(slate_diversity)
print(diversity)

In [ ]:
print("Is diversity equal? ", measurements['diversity_metric'][-1] == diversity)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
# Getting all possible 2-item combinations (the indices) for the items in a slate
combos = combinations(np.arange(bubble.num_items_per_iter), 2)
items_shown = bubble.items_shown

topic_similarity = np.zeros(bubble.num_users)

stop = 0
for i in combos:
    # topic_similarity is equal to the number of 2-item combinations in which the items' topics are the same
    item_pair = items_shown[:, i]
    topic_pair = bubble.item_topics[item_pair]
    topic_similarity += (topic_pair[:,0] == topic_pair[:,1])

slate_diversity = 1 - ((1 / (bubble.num_items_per_iter * (bubble.num_items_per_iter-1))) * topic_similarity)
diversity = np.mean(slate_diversity)
print(diversity)

In [ ]:
print("Is diversity equal? ", measurements['diversity_metric'][-1] == diversity)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

In [ ]:
# Getting all possible 2-item combinations (the indices) for the items in a slate
combos = combinations(np.arange(bubble.num_items_per_iter), 2)
items_shown = bubble.items_shown

topic_similarity = np.zeros(bubble.num_users)

stop = 0
for i in combos:
    # topic_similarity is equal to the number of 2-item combinations in which the items' topics are the same
    item_pair = items_shown[:, i]
    topic_pair = bubble.item_topics[item_pair]
    topic_similarity += (topic_pair[:,0] == topic_pair[:,1])

slate_diversity = 1 - ((1 / (bubble.num_items_per_iter * (bubble.num_items_per_iter-1))) * topic_similarity)
diversity = np.mean(slate_diversity)
print(diversity)

In [ ]:
print("Is diversity equal? ", measurements['diversity_metric'][-1] == diversity)

In [ ]:
for key in bubble.get_measurements().keys():
    print(key, bubble.get_measurements()[key])

# Testing **all** evalmetrics

### Repeates **not** allowed

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
from wrapper.metrics import NoveltyMetric, SerendipityMetric, DiversityMetric

bubble.add_metrics(MSEMeasurement(), NoveltyMetric(), SerendipityMetric(), DiversityMetric())
print("These are the current metrics:")
print(bubble.metrics)

In [ ]:
bubble.run(timesteps=1)
measurements = bubble.get_measurements()

for key in measurements:
    print(key, "-", measurements[key])

In [ ]:
bubble.run(timesteps=100)
measurements = bubble.get_measurements()

# print("Measurements for iters: ", str(np.arange(11) * 10))
for key in measurements:
    print(key, "\n\t", measurements[key][1::10])

### Repeates **allowed**

In [ ]:
bubble = BubbleBurster(
    # num_users=number_of_users,
    # num_items=num_items,
    # num_attributes=number_of_attributes,
    item_topics=item_topics,
    user_representation=user_representation,
    item_representation=item_representation,
    # actual_user_representation=users,
    record_base_state=True,
)

In [ ]:
from wrapper.metrics import NoveltyMetric, SerendipityMetric, DiversityMetric

bubble.add_metrics(MSEMeasurement(), NoveltyMetric(), SerendipityMetric(), DiversityMetric())
print("These are the current metrics:")
print(bubble.metrics)

In [ ]:
bubble.run(timesteps=100)
measurements = bubble.get_measurements()

# print("Measurements for iters: ", str(np.arange(11) * 10))
for key in measurements:
    print(key, "\n\t", measurements[key][1::10])